In [7]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-267508
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-267508


In [11]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "cpu-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0, # when innactive
                                                           max_nodes=4) # when busy

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

Found existing cpu-cluster


In [18]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': choice(0.01, 0.1, 0.2, 0.5, 0.7, 1.0),
    '--max_iter': choice(range(10,110,10))
    }
) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, 
                      evaluation_interval=2, 
                      delay_evaluation=5)  ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig( 
    source_directory='./',
    compute_target=cpu_cluster, 
    script='train.py',
    environment=sklearn_env
) ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     primary_metric_name="Accuracy", 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     max_total_runs=10, 
                                     max_concurrent_runs=5) ### YOUR CODE HERE ###

In [19]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hypdrive_run = exp.submit(hyperdrive_config)

RunDetails(hypdrive_run).show()
hypdrive_run.wait_for_completion(show_output=True)

In [26]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hypdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])

best_run

Best Run Id:  HD_1ff1ba95-d1c8-4da7-809e-2aab27fb7bc7_4
Accuracy: 0.9111785533636824


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_1ff1ba95-d1c8-4da7-809e-2aab27fb7bc7_4,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


# AutoML Run

In [1]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory

ds = TabularDatasetFactory.from_delimited_files(path = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

In [2]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [3]:
from sklearn.model_selection import train_test_split

# split data into train and test sets - 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=123, stratify = y)

In [4]:
import pandas as pd

# Create a new dataframe with only training data

df_train = pd.concat([x_train,y_train], axis=1)
df_train.reset_index(drop=True, inplace=True)

In [5]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=df_train,
    label_column_name='y',
    n_cross_validations=5)

In [20]:
!pip install -r /anaconda/envs/azureml_py38/lib/python3.9/site-packages/azureml/automl/core/validated_linux_requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 51.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.3
    Uninstalling scipy-1.5.3:
      Successfully uninstalled scipy-1.5.3
  Attempting uninstall: azure-mgmt-resource
    Found existing installation: azure-mgmt-resource 23.1.0b2
    Uninstalling azure-mgmt-resource-23.1.0b2:
      Successfully uninstalled azure-mgmt-resource-23.1.0b2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.22.0 requires networkx>=2.8, but you have networkx 2.5 which is incompatible.
responsibleai 0.33.0 requires ipykernel<=6.8.0, but you have ipykernel 6.29.4 which is incompatible.
econml 0.15.0 requires shap<0.44.0,>=0.38.1, but you have shap 0.44.0 which is incompatible

In [8]:
# Submit your automl run

experiment_name = 'automl-experiment'

experiment = Experiment(ws, experiment_name)
automl_run = experiment.submit(config=automl_config, show_output=True)
automl_run

No run_configuration provided, running on local with default configuration
Running in the active local environment.


2024-09-11 18:34:09.950145: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-09-11 18:34:09.950174: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-09-11 18:34:19.357112: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-09-11 18:34:24.594612: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-09-11 18:34:24.594723: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (quyencompute): /proc/driver/nvidia/version does not exist


Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_ee8ef12e-21d2-49bc-bd94-9412e195b870,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2024/09/11 18:39:44 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024-09-11:19:05:54,241 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2024-09-11:19:05:55,103 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2024-09-11:19:05:55,104 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2024-09-11:19:05:55,979 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2024-09-11:

Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_ee8ef12e-21d2-49bc-bd94-9412e195b870,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [13]:
# from azureml.widgets import RunDetails
RunDetails(automl_run).show()

automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_ee8ef12e-21d2-49bc-bd94-9412e195b870,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2584                          |1                               |23065                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_ee8ef12e-21d2-49bc-bd94-9412e195b870',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2024-09-11T18:34:37.62252Z',
 'endTimeUtc': '2024-09-11T19:06:09.442028Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-experiment","subscription_id":"3e42d11f-d64d-4173-af9b-12ecaa1030b3","resource_group":"aml-quickstarts-267508","workspace_name":"quick-starts-ws-267508","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1

In [14]:
# Retrieve model

best_run, fitted_model = automl_run.get_output()

# get name of files of best_run
best_run.get_file_names()

['accuracy_table',
 'confusion_matrix',
 'explanation/bd39664b/classes.interpret.json',
 'explanation/bd39664b/eval_data_viz.interpret.json',
 'explanation/bd39664b/expected_values.interpret.json',
 'explanation/bd39664b/features.interpret.json',
 'explanation/bd39664b/global_names/0.interpret.json',
 'explanation/bd39664b/global_rank/0.interpret.json',
 'explanation/bd39664b/global_values/0.interpret.json',
 'explanation/bd39664b/local_importance_values.interpret.json',
 'explanation/bd39664b/per_class_names/0.interpret.json',
 'explanation/bd39664b/per_class_rank/0.interpret.json',
 'explanation/bd39664b/per_class_values/0.interpret.json',
 'explanation/bd39664b/rich_metadata.interpret.json',
 'explanation/bd39664b/true_ys_viz.interpret.json',
 'explanation/bd39664b/visualization_dict.interpret.json',
 'explanation/bd39664b/ys_pred_proba_viz.interpret.json',
 'explanation/bd39664b/ys_pred_viz.interpret.json',
 'explanation/f2740280/classes.interpret.json',
 'explanation/f2740280/expe

In [15]:
# save best model
best_run.register_model(model_name = "model.pkl", model_path = './outputs/')
print(fitted_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('24', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.625, reg_lambda=0.8333333333333334, subsample=0.8, tree_method='auto'))], verbose=False)), ('14', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.3, gamma=0, max_depth=10, max_leaves=511, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=

# Cleaning Up Cluster

In [ ]:
cpu_cluster.delete()